In [1]:
import pandas as pd
import numpy  as np
import os

In [2]:
SPLUS_Columns = ['ID', 'RA', 'DEC']
Data          = pd.read_csv('/home/erik/Pesquisa/SPLUS/DownloadData/SPLUS_iDR4.3_AllFeatures_Spec_20221206.csv', usecols=SPLUS_Columns)

Data[SPLUS_Columns].to_csv('TestData.csv', index=False)

VHS_columns_to_keep    = ['SrcID', 'Ypmag', 'e_Ypmag', 'Jpmag', 'e_Jpmag', 'Hpmag', 'e_Hpmag', 'Kspmag', 'e_Kspmag', 'angDist']
GALEX_columns_to_keep  = ['objid', 'FUVmag', 'e_FUVmag', 'NUVmag', 'e_NUVmag', 'angDist']
unWISE_columns_to_keep = ['objID', 'FW1', 'e_FW1', 'FW2', 'e_FW2', 'angDist']

## Match com VHS
Columns_to_keep = ' '.join(SPLUS_Columns+VHS_columns_to_keep)

os.system(f"""java -jar stilts.jar cdsskymatch in=TestData.csv cdstable=II/367/vhs_dr5 ra=RA dec=DEC radius=1 find=each blocksize=100000 \\
              ocmd='keepcols "{Columns_to_keep}"; addcol ID_VHS "SrcID"; addcol sep_VHS "angDist"; delcols "SrcID angDist"' out=TestData_Matched.csv""")

# Get new columns to keep
Data              = pd.read_csv('TestData_Matched.csv', nrows=1)
SPLUS_VHS_to_keep = list(Data.columns.values)

## Match with GALEX
Columns_to_keep = ' '.join(SPLUS_VHS_to_keep+GALEX_columns_to_keep)

os.system(f"""java -jar stilts.jar cdsskymatch in=TestData_Matched.csv cdstable=II/335/galex_ais ra=RA dec=DEC radius=2 find=each blocksize=100000 \\
              ocmd='keepcols "{Columns_to_keep}"; addcol ID_GALEX "objid"; addcol sep_GALEX "angDist"; delcols "objid angDist"' out=TestData_Matched_2.csv""")

# Get new columns to keep
Data                    = pd.read_csv('TestData_Matched_2.csv', nrows=1)
SPLUS_VHS_GALEX_to_keep = list(Data.columns.values)

## Match with unWISE
Columns_to_keep = ' '.join(SPLUS_VHS_GALEX_to_keep+unWISE_columns_to_keep)

os.system(f"""java -jar stilts.jar cdsskymatch in=TestData_Matched_2.csv cdstable=II/363/unwise ra=RA dec=DEC radius=1 find=each blocksize=100000 \\
              ocmd='keepcols "{Columns_to_keep}"; addcol ID_unWISE "objID"; addcol sep_unWISE "angDist"; delcols "objID angDist"' out=TestData_Matched_3.csv""")

0

In [3]:
Data = pd.read_csv('TestData_Matched_3.csv')

In [4]:
Final_Columns = ['ID', 'RA', 'DEC',
                 'ID_VHS', 'Ypmag', 'Jpmag', 'Hpmag', 'Kspmag', 'e_Ypmag', 'e_Jpmag', 'e_Hpmag', 'e_Kspmag', 'sep_VHS', 
                 'ID_GALEX', 'FUVmag', 'NUVmag', 'e_FUVmag', 'e_NUVmag', 'sep_GALEX', 
                 'ID_unWISE', 'W1_ab', 'W2_ab', 'e_W1_ab', 'e_W2_ab', 'sep_unWISE']

In [5]:
Data['W1_ab'] = 22.5 - 2.5*np.log10(Data['FW1']) + 2.699
Data['W2_ab'] = 22.5 - 2.5*np.log10(Data['FW2']) + 3.339

# http://doxygen.lsst.codes/stack/doxygen/x_masterDoxyDoc/namespacelsst_1_1afw_1_1image.html#a0a023f269211d52086723788764d484e
# return std::abs(fluxErr / (-0.4 * flux * std::log(10)));
Data['e_W1_ab'] = abs(Data['e_FW1'] / (-0.4*Data['FW1']*np.log(10)))
Data['e_W2_ab'] = abs(Data['e_FW2'] / (-0.4*Data['FW2']*np.log(10)))

Data['W1_ab'].replace([np.inf, -np.inf], np.nan, inplace=True)
Data['W2_ab'].replace([np.inf, -np.inf], np.nan, inplace=True)

#Data[Final_Columns].to_csv('SPLUS_iDR4.3_VHS_GAL_unW_Photometry_20221125.csv', index=False)

/home/erik/Pesquisa/Anaconda3/envs/TFGPU/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/erik/Pesquisa/Anaconda3/envs/TFGPU/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [6]:
# Calcualte nDets
Base_Filters   = ['u', 'J0378', 'J0395', 'J0410', 'J0430', 'g', 'J0515', 'r', 'J0660', 'i', 'J0861', 'z']
Apertures      = ['aper_3', 'aper_6', 'petro', 'iso', 'auto', 'PStotal']

for aper in Apertures:
    Features = [s+'_'+aper for s in Base_Filters]
    nDetData = pd.read_csv('/home/erik/Pesquisa/SPLUS/DownloadData/SPLUS_iDR4.3_AllFeatures_Spec_20221206.csv', usecols=Features)
    Data['nDet_'+aper] = (12-(nDetData == 99).sum(axis=1))

In [7]:
import extinction
import sfdmap

m = sfdmap.SFDMap('DustMaps/')

Base_Filters   = ['u', 'J0378', 'J0395', 'J0410', 'J0430', 'g', 'J0515', 'r', 'J0660', 'i', 'J0861', 'z']
Extinct_SPLUS  = [filt+'_ext' for filt in Base_Filters]

def correct_extinction(Dataframe, Extinction_Maps):
    '''
    Correct the magnitudes for extinction using the CCM89 Law

    Keyword arguments:
    Dataframe         -- Dataframe containing the data to be corrected
    Extinction_Maps   -- SFD Maps
    '''
    Corrected_Dataframe = Dataframe.copy().reset_index(drop=True)

    # Obtaining E(B-V) and Av in a given RA, DEC position
    AV = Extinction_Maps.ebv(Dataframe.RA, Dataframe.DEC)*3.1

    # Calculating the extinction on the S-PLUS bands using the Cardelli, Clayton & Mathis law.
    Lambdas = np.array([3536, 3770, 3940, 4094, 4292, 4751, 5133, 6258, 6614, 7690, 8611, 8831]).astype(np.float)

    Extinctions = []
    for i in range(len(AV)):
        Extinctions.append(extinction.ccm89(Lambdas, AV[i], 3.1))

    Extinction_DF = pd.DataFrame(Extinctions, columns=Extinct_SPLUS)

    for i in range(len(Extinct_SPLUS)):
        Corrected_Dataframe[Extinct_SPLUS[i]] = Extinction_DF[Extinct_SPLUS[i]]
        
    Corrected_Dataframe['EBV'] = AV/3.1

    return Corrected_Dataframe

In [8]:
nDetCols = ['nDet_'+s for s in Apertures]

In [9]:
Corrected_DF = correct_extinction(Data, m)
Corrected_DF[Final_Columns+Extinct_SPLUS+['EBV']+nDetCols].to_csv('SPLUS_iDR4.3_VHS_GAL_unW_Extinctions_nDets_20221206.csv', index=False)

/home/erik/Pesquisa/Anaconda3/envs/TFGPU/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [11]:
os.system('rm TestData.csv')
os.system('rm TestData_Matched.csv')
os.system('rm TestData_Matched_2.csv')
os.system('rm TestData_Matched_3.csv')

0